# 数据可视化

In [ ]:
# 读取数据
import pandas as pd

world_data = pd.read_csv('today_world_2021_01_07.csv')

import pyecharts
# 调整配置项
import pyecharts.options as opts
# Map类用于绘制地图
from pyecharts.charts import Map

In [ ]:
#利用各国家的累计确诊人数减去累计治愈和累计死亡人数得到现存确诊人数，作为新的一列特征添加到数据中：
world_data['today_storeConfirm'] = world_data['total_confirm'] - world_data['total_heal'] - world_data['total_dead']
world_data.head()

,id,lastUpdateTime,name,total_confirm,total_dead,total_heal,total_input,total_severe,total_suspect,today_confirm,today_dead,today_heal,today_input,today_severe,today_storeConfirm,today_suspect
0,9577772,2021-01-07 08:23:02,突尼斯,149881,5004,111599,0.0,0,0,2820.0,70.0,1275.0,NaN,NaN,33278,NaN
1,9507896,2021-01-07 08:23:02,塞尔维亚,352120,3444,0,0.0,0,0,2882.0,0.0,0.0,NaN,NaN,348676,NaN
2,0,2021-01-07 16:38:30,中国,97218,4795,91107,4359.0,0,0,91.0,1.0,99.0,11.0,NaN,1316,NaN
3,1,2021-01-07 15:58:28,日本本土,262809,3617,206883,0.0,0,0,3314.0,8.0,0.0,NaN,NaN,52309,NaN
4,2,2021-01-07 14:23:37,泰国,9636,67,4521,0.0,0,0,305.0,1.0,103.0,NaN,NaN,5048,NaN


In [ ]:
#载入国家中英文对照表，使用Series类的replace方法将各国家对应的中文名称替换成英文名称：
contry_name = pd.read_csv('county_china_english.csv', encoding='GB2312')
contry_name.head()

,英文,中文
0,Afghanistan,阿富汗
1,Aland Islands,奥兰群岛
2,Albania,阿尔巴尼亚
3,Algeria,阿尔及利亚
4,American Samoa,美属萨摩亚


In [ ]:
world_data['eg_name'] = world_data['name'].replace(contry_name['中文'].values ,contry_name['英文'].values)
world_data['eg_name'].head()

0     Tunisia
1      Serbia
2       China
3        日本本土
4    Thailand
Name: eg_name, dtype: object

In [ ]:
#提取出我们需要的数据，保存成一个嵌套列表的形式：
heatmap_data = world_data[['eg_name','today_storeConfirm']].values.tolist()
heatmap_data[:10]

[['Tunisia', 33278],
 ['Serbia', 348676],
 ['China', 1316],
 ['日本本土', 52309],
 ['Thailand', 5048],
 ['Singapore', 210],
 ['Korea', 17991],
 ['Australia', 1827],
 ['Germany', 307211],
 ['United States', 8463484]]

In [ ]:
#接下来我们开始绘图，首先初始化类对象Map，并调用add方法添加绘图基本配置：
map_ = Map().add(series_name = "现存确诊人数", # 设置提示框标签
                 data_pair = heatmap_data, # 输入数据
                 maptype = "world", # 设置地图类型为世界地图
                 is_map_symbol_show = False # 不显示标记点
                ) 

In [ ]:
# 设置系列配置项
map_.set_series_opts(label_opts=opts.LabelOpts(is_show=False))  # 不显示国家（标签）名称

In [ ]:
# 设置全局配置项
map_.set_global_opts(title_opts = opts.TitleOpts(title="世界各国家现存确诊人数地图"), # 设置图标题
                     # 设置视觉映射配置项
                     visualmap_opts = opts.VisualMapOpts(pieces=[ # 自定义分组的分点和颜色
                                                               {"min": 10000,"color":"#800000"}, # 栗色
                                                               {"min": 5000, "max": 9999, "color":"#B22222"}, # 耐火砖
                                                               {"min": 999, "max": 4999,"color":"#CD5C5C"}, # 印度红
                                                               {"min": 100, "max": 999, "color":"#BC8F8F"}, # 玫瑰棕色
                                                               {"max": 99, "color":"#FFE4E1"}, # 薄雾玫瑰
                                                              ], 
                     is_piecewise = True))  # 显示分段式图例

In [ ]:
# 在notebook中进行渲染
map_.render_notebook()

# 世界国家累计死亡人数玫瑰图

In [ ]:
#首先筛选出累计死亡人数超过500人的世界国家，并按人数进行降序排序：
need_data = world_data[['name','total_dead']][world_data['total_dead'] >500]

rank = need_data[['name','total_dead']].sort_values(by='total_dead',ascending=False).values

In [ ]:
#导入Pie类并添加绘图的基本配置
from pyecharts.charts import Pie

pie = Pie().add("累计死亡人数", # 添加提示框标签
                rank, # 输入数据
                radius = ["20%", "80%"],  # 设置内半径和外半径
                center = ["60%", "60%"],  # 设置圆心位置
                rosetype = "radius")   # 玫瑰图模式，通过半径区分数值大小，角度大小表示占比

In [ ]:
#设置全局和系列配置项并绘制玫瑰图：
pie.set_global_opts(title_opts = opts.TitleOpts(title="世界国家累计死亡人数玫瑰图",  # 设置图标题
                                                pos_right = '40%'),  # 图标题的位置
                    legend_opts = opts.LegendOpts( # 设置图例
                                                orient='vertical', # 垂直放置图例
                                                pos_right="85%", # 设置图例位置
                                                pos_top="15%"))

pie.set_series_opts(label_opts = opts.LabelOpts(formatter="{b} : {d}%")) # 设置标签文字形式为（国家：占比（%））
# 在notebook中进行渲染     
pie.render_notebook()

# 3月世界国家累计确诊人数动态条形图

In [ ]:
alltime_data = pd.read_csv('alltime_world_2021_01_07.csv')

In [ ]:
#挑选出疫情最为严重的10个国家，并筛选出这些国家的历史疫情数据：
country_list = ['美国', '意大利', '中国', '西班牙', '德国', '伊朗', '法国', '英国', '瑞士','比利时']
need_data = alltime_data[alltime_data['name'].isin(country_list)]

In [ ]:
#使用datetime模块生成时间数据，构造时间列表：
from datetime import datetime,timedelta
time_list = [(datetime(2020, 3, 1) + timedelta(i)).strftime('%Y-%m-%d') for i in range(31)]

In [ ]:
#载入Matplotlib库，并设置正常显示中文字体：
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['figure.dpi'] = 100

In [ ]:
#为每个国家设置一种颜色：
color_list = ['brown','peru','orange','blue','green',
              'red','yellow','teal','pink','orchid']
country_color = pd.DataFrame()
country_color['country'] = country_list
country_color['color'] = color_list

In [ ]:
#定义绘图函数：
import matplotlib.ticker as ticker

def barh_draw(day):
    
    # 提取每一天的数据
    draw_data = need_data[need_data['date']==day][['name','total_confirm']].sort_values(by='total_confirm',ascending=True)
    
    # 清空当前的绘图
    ax.clear()
    
    # 绘制条形图
    ax.barh(draw_data['name'],draw_data['total_confirm'], color=[country_color[country_color['country']==i]['color'].values[0] for i in draw_data['name']])
    
    # 数值标签的间距
    dx = draw_data['total_confirm'].max()/200
    
    # 添加数值标签
    for j, (name, value) in enumerate(zip(draw_data['name'], draw_data['total_confirm'])):
        
        ax.text(value+dx, j, f'{value:,.0f}', size=10, ha='left', va='center')
        
    # 添加日期标签
    ax.text(draw_data['total_confirm'].max()*0.75, 0.4, day, color='#777777',size=40, ha='left')
    
    # 设置刻度标签的格式
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    
    # 设置刻度的位置
    ax.xaxis.set_ticks_position('top')

    # 设置刻度标签的颜色和大小
    ax.tick_params(axis='x',colors='#777777', labelsize=15)
    
    # 添加网格线
    ax.grid(which='major',axis='x',linestyle='-')
    
    # 添加图标题
    ax.text(0, 11, '3月世界各国家累计确诊人数动态条形图',size=20, ha='left')
    
    # 去除图边框
    plt.box(False)
    
    # 关闭绘图框
    plt.close()

# 动态绘图
fig, ax = plt.subplots(figsize=(12, 8))

import matplotlib.animation as animation
from IPython.display import HTML

animator = animation.FuncAnimation(fig, barh_draw, frames=time_list, interval=200)
HTML(animator.to_jshtml())